# The Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('insurance_claims.csv')

In [3]:
df.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital-gains', 'capital-loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported', '_c39'],
      dtype='object')

In [4]:
df.shape

(1000, 40)

In [5]:
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


In [6]:
# rename column; fix typo

df.rename(columns = {'policy_deductable':'policy_deductible'}, inplace = True)

# Data Cleaning

## Check for irregular records

**Check for NA values**

In [7]:
df.isna().sum()

months_as_customer                0
age                               0
policy_number                     0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductible                 0
policy_annual_premium             0
umbrella_limit                    0
insured_zip                       0
insured_sex                       0
insured_education_level           0
insured_occupation                0
insured_hobbies                   0
insured_relationship              0
capital-gains                     0
capital-loss                      0
incident_date                     0
incident_type                     0
collision_type                    0
incident_severity                 0
authorities_contacted             0
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_damage             

**Drop duplicates (if any)**

In [8]:
df = df.drop_duplicates()

## Check columns

**umbrella_limit**

In [9]:
print(sorted(df['umbrella_limit'].unique()))

df[df['umbrella_limit'] == -1000000]

# Found 1 record with negative value for umbrella_limit.

[-1000000, 0, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000, 9000000, 10000000]


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductible,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
290,284,42,526039,1995-05-04,OH,100/300,500,1338.54,-1000000,438178,...,?,94160,8560,17120,68480,Chevrolet,Malibu,1996,N,NaN


In [10]:
# Assume negative umbrella_limit is a typo and change to positive value.

df.loc[df['umbrella_limit'] == -1000000, 'umbrella_limit'] = 1000000

print(sorted(df['umbrella_limit'].unique()))

[0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000, 9000000, 10000000]


**insured_zip**

In [11]:
df['insured_zip']

# Has 6 digits. Will drop because data is not usable.

0      466132
1      468176
2      430632
3      608117
4      610706
        ...  
995    431289
996    608177
997    442797
998    441714
999    612260
Name: insured_zip, Length: 1000, dtype: int64

**insured_relationship**

In [12]:
df['insured_relationship'].unique()

# Data is too disparate. Will drop because not enough explanation.

array(['husband', 'other-relative', 'own-child', 'unmarried', 'wife',
       'not-in-family'], dtype=object)

**capital-gains, capital-loss**

In [13]:
df[['capital-gains', 'capital-loss']]

# No explanation of meaning of these fields. Will drop.

,capital-gains,capital-loss
0,53300,0
1,0,0
2,35100,0
3,48900,-62400
4,66000,-46000
...,...,...
995,0,0
996,70900,0
997,35100,0
998,0,0


**collision_type**

In [14]:
df['collision_type'].unique()

array(['Side Collision', '?', 'Rear Collision', 'Front Collision'],
      dtype=object)

In [15]:
# Check if meaning of '?' means 'no collision'

df['incident_type'][df['collision_type'] == '?'].unique()

array(['Vehicle Theft', 'Parked Car'], dtype=object)

In [16]:
# Change '?' of collision_type into 'N/A'

df.loc[df['collision_type'] == '?', 'collision_type'] = 'N/A'

df['collision_type'].unique()

array(['Side Collision', 'N/A', 'Rear Collision', 'Front Collision'],
      dtype=object)

**incident_location**

In [17]:
df['incident_location']

# Too granular. Will drop.

0           9935 4th Drive
1             6608 MLK Hwy
2        7121 Francis Lane
3         6956 Maple Drive
4             3041 3rd Ave
              ...         
995     6045 Andromedia St
996       3092 Texas Drive
997            7629 5th St
998          6128 Elm Lane
999    1416 Cherokee Ridge
Name: incident_location, Length: 1000, dtype: object

**property_damage**

In [18]:
df['property_damage'].unique()

array(['YES', '?', 'NO'], dtype=object)

In [19]:
sum(df['property_damage'] == '?')

# Too many '?', and no information on it. Will drop.

360

**police_report_available**

In [20]:
df['police_report_available'].unique()

array(['YES', '?', 'NO'], dtype=object)

In [21]:
sum(df['police_report_available'] == '?')

# Too many '?', and no information on it. Will drop.

343

**auto_model**

In [22]:
df['auto_model'].unique()

# Too granular. Will drop.

array(['92x', 'E400', 'RAM', 'Tahoe', 'RSX', '95', 'Pathfinder', 'A5',
       'Camry', 'F150', 'A3', 'Highlander', 'Neon', 'MDX', 'Maxima',
       'Legacy', 'TL', 'Impreza', 'Forrestor', 'Escape', 'Corolla',
       '3 Series', 'C300', 'Wrangler', 'M5', 'X5', 'Civic', 'Passat',
       'Silverado', 'CRV', '93', 'Accord', 'X6', 'Malibu', 'Fusion',
       'Jetta', 'ML350', 'Ultima', 'Grand Cherokee'], dtype=object)

**_c39**

In [23]:
df['_c39'].unique()

# Extra column. Will drop.

array([nan])

## Drop Columns

In [24]:
df = df.drop(['insured_zip', 'insured_relationship', 'capital-gains', 'capital-loss', 'incident_location', 
         'property_damage', 'police_report_available', 'auto_model', '_c39'], axis = 1)

## Final Dataframe

In [25]:
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductible,policy_annual_premium,umbrella_limit,insured_sex,...,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,MALE,...,1,1,2,71610,6510,13020,52080,Saab,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,MALE,...,1,0,0,5070,780,780,3510,Mercedes,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,FEMALE,...,3,2,3,34650,7700,3850,23100,Dodge,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,FEMALE,...,1,1,2,63400,6340,6340,50720,Chevrolet,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,MALE,...,1,0,1,6500,1300,650,4550,Accura,2009,N


In [26]:
df.shape

(1000, 31)

In [27]:
df.columns

Index(['months_as_customer', 'age', 'policy_number', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductible',
       'policy_annual_premium', 'umbrella_limit', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim', 'auto_make', 'auto_year',
       'fraud_reported'],
      dtype='object')

In [28]:
df.to_csv('insurance_claims_cleaned.csv')

# Summary

In [29]:
print('months_as_customer:', min(df['months_as_customer']), max(df['months_as_customer']))
print('age:', min(df['age']), max(df['age']))
print('policy_bind_date:', min(df['policy_bind_date']), max(df['policy_bind_date']))
print('policy_state:', df['policy_state'].unique())
print('policy_csl:', sorted(df['policy_csl'].unique()))
print('policy_deductible:',sorted(df['policy_deductible'].unique()))
print('policy_annual_premium:', min(df['policy_annual_premium']), max(df['policy_annual_premium']))
print('umbrella_limit:', sorted(df['umbrella_limit'].unique()))
print('insured_sex:', df['insured_sex'].unique())
print('insured_education_level:', df['insured_education_level'].unique())
print('insured_occupation:', df['insured_occupation'].unique())
print('insured_hobbies:',df['insured_hobbies'].unique())
print('incident_date:', min(df['incident_date']), max(df['incident_date']))
print('incident_type:', df['incident_type'].unique())
print('collision_type:', df['collision_type'].unique())
print('authorities_contacted:', df['authorities_contacted'].unique())
print('incident_state:', df['incident_state'].unique())
print('incident_city:', df['incident_city'].unique())
print('incident_hour_of_the_day:', sorted(df['incident_hour_of_the_day'].unique()))
print('number_of_vehicles_involved:', min(df['number_of_vehicles_involved']), max(df['number_of_vehicles_involved']))
print('bodily_injuries:', min(df['bodily_injuries']), max(df['bodily_injuries']))
print('witnesses:', min(df['witnesses']), max(df['witnesses']))
print('total_claim_amount:', min(df['total_claim_amount']), max(df['total_claim_amount']))
print('injury_claim:', min(df['injury_claim']), max(df['injury_claim']))
print('property_claim:', min(df['property_claim']), max(df['property_claim']))
print('vehicle_claim', min(df['vehicle_claim']), max(df['vehicle_claim']))
print('auto_make:', df['auto_make'].unique())
print('auto_year:', min(df['auto_year']), max(df['auto_year']))
print('fraud_reported:', df['fraud_reported'].unique())

months_as_customer: 0 479
age: 19 64
policy_bind_date: 1990-01-08 2015-02-22
policy_state: ['OH' 'IN' 'IL']
policy_csl: ['100/300', '250/500', '500/1000']
policy_deductible: [500, 1000, 2000]
policy_annual_premium: 433.33 2047.59
umbrella_limit: [0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000, 9000000, 10000000]
insured_sex: ['MALE' 'FEMALE']
insured_education_level: ['MD' 'PhD' 'Associate' 'Masters' 'High School' 'College' 'JD']
insured_occupation: ['craft-repair' 'machine-op-inspct' 'sales' 'armed-forces' 'tech-support'
 'prof-specialty' 'other-service' 'priv-house-serv' 'exec-managerial'
 'protective-serv' 'transport-moving' 'handlers-cleaners' 'adm-clerical'
 'farming-fishing']
insured_hobbies: ['sleeping' 'reading' 'board-games' 'bungie-jumping' 'base-jumping' 'golf'
 'camping' 'dancing' 'skydiving' 'movies' 'hiking' 'yachting' 'paintball'
 'chess' 'kayaking' 'polo' 'basketball' 'video-games' 'cross-fit'
 'exercise']
incident_date: 2015-01-01 2015-03-01


Assume dataset was updated as of March 1st, 2015.

Policies are about incidents from January 1st to March 1st, 2015.

**This cleaned dataset was used to create the Power BI report on the GitHub Repo.**

**Soon this notebook will be updated to include the ML portion.**